In [1]:
import time
from pyspark.sql import SparkSession
import psutil
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, MapType
import pandas as pd
from functools import reduce
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col
from pyspark import StorageLevel

In [2]:
# importing costum module , don't need both but i have both lol 
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "./src/")))

ROOT_DIR = "."
SRC_DIR = ROOT_DIR + "/src"
sys.path.append("/Users/admin/projectst/EEG-Feature-Benchmark")

In [3]:
 if SparkContext._active_spark_context:
        print("Stopping existing Spark context...")
        SparkContext._active_spark_context.stop()
        print("Previous Spark context stopped successfully")

In [4]:
spark = (
        SparkSession.builder
        .appName("EEG_Analysis_HPC")

        # Target ~12 executors across the cluster (safe parallelism)
        .config("spark.executor.instances", "12")
        .config("spark.executor.cores", "1")             # 1 core per executor
        .config("spark.executor.memory", "4g")           # 12 × 4g = 48 GB total
        .config("spark.driver.memory", "6g")             # leave room for driver ops

        # Parallelism and shuffle tuning
        .config("spark.default.parallelism", "12")
        .config("spark.sql.shuffle.partitions", "12")

        # UDF / pandas optimization
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config("spark.sql.execution.arrow.maxRecordsPerBatch", "500")

        # JVM GC tuning
        .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:+UseStringDeduplication")

        # Optional: Raise broadcast size if EEG config or schema is large
        .config("spark.sql.autoBroadcastJoinThreshold", "20MB")

        .getOrCreate()
    )

25/05/04 20:31:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 20:31:15 WARN Utils: Service 'SparkUI' could not bind on port 57690. Attempting port 57691.


In [5]:
print("New Spark session created successfully")
print("=== Spark Configuration & Runtime info ===")
print(f"App ID: {spark.sparkContext.applicationId}")
print(f"Master: {spark.sparkContext.master}")
print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")
print(f"Total Executors: {spark.sparkContext._jsc.sc().getExecutorMemoryStatus().size()}")

for item in spark.sparkContext.getConf().getAll():
    print(f"{item[0]} = {item[1]}")
    

New Spark session created successfully
=== Spark Configuration & Runtime info ===
App ID: app-20250504203115-0002
Master: spark://cm001:23412
Default Parallelism: 12
Total Executors: 1
spark.default.parallelism = 12
spark.sql.execution.arrow.maxRecordsPerBatch = 500
spark.executor.instances = 12
spark.executor.extraJavaOptions = -XX:+UseG1GC -XX:+UseStringDeduplication
spark.sql.shuffle.partitions = 12
spark.ui.killEnabled = false
spark.driver.port = 45357
spark.driver.memory = 6g
spark.ui.proxyBase = /rnode/cm001.hpc.nyu.edu/57690
spark.serializer.objectStreamReset = 100
spark.master = spark://cm001:23412
spark.submit.deployMode = client
spark.app.startTime = 1746405074396
spark.sql.execution.arrow.pyspark.enabled = true
spark.driver.host = cm001.hpc.nyu.edu
spark.executor.memory = 4g
spark.ui.enabled = true
spark.sql.warehouse.dir = file:/home/ans9868/EEG-HPC-benchmark/spark-warehouse
spark.executor.id = driver
spark.app.id = app-20250504203115-0002
spark.sql.autoBroadcastJoinThresho